In [1]:
import requests  # 发送请求
import random
from time import sleep  # 设置等待，防止反爬
import time
import pandas as pd  # 保存csv
import datetime
import os
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup


In [2]:
# # 尝试使用用户代理池来模拟真实的浏览器请求
# # 在此列表中添加多个用户代理
# USER_AGENTS = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
#     "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
#     "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
#     "Opera/9.80 (Windows NT 6.1; WOW64) Presto/2.12.388 Version/12.18",
#     "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
#     "Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1"
# ]

# MY_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"


In [3]:
# #singapore travel
# hxs_url = "https://www.xiaohongshu.com/web_api/sns/v3/page/notes?page_size=6&sort=hot&page_id=5c0b4408944088000107a919&cursor=&sid="

# #singapore hotel
# hxs_url = "https://www.xiaohongshu.com/web_api/sns/v3/page/notes?page_size=6&sort=hot&page_id=5a438d97800086066172013f&cursor=&sid="

# #singapore food
# hxs_url = "https://www.xiaohongshu.com/web_api/sns/v3/page/notes?page_size=6&sort=hot&page_id=5bf7aed3ed307f0001beb4ea&cursor=&sid="



                        
# # parse URL and get page id
# parsed_url = urlparse(hxs_url)
# query_params = parse_qs(parsed_url.query)
# page_id = query_params.get('page_id', [None])[0] 
# print(page_id)

In [4]:
hxs_topic_page_id_list = {
    'guide': '5c0b4408944088000107a919',
    'hotel': '5a438d97800086066172013f',
    'food': '5bf7aed3ed307f0001beb4ea',
    'tour': '5a438d92800086066171fa4a',
    'play': '62836da8a1b02800018c8be7'
}

TODAY_YMD = datetime.datetime.now().strftime("%Y-%m-%d")


In [5]:
#TODO: page_url -> note_url
#TODO: add userid column to user df

def hxs_post_scrapper(itemtype, itemcount=600, timeout_max_sec=3):
    
    #set user agents
    USER_AGENTS = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0",
        "Opera/9.80 (Windows NT 6.1; WOW64) Presto/2.12.388 Version/12.18",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1"
    ]
    h1 = {
        # 'User-Agent': MY_AGENT
        "User-Agent": random.choice(USER_AGENTS),
    }
    
    #set topic page id
    page_id = hxs_topic_page_id_list[itemtype]
    
    #init params
    pagesize = 6
    page = 1
    results=[]
    comments=[]
    users ={}
    

    try:
        while True: 
            if page==1:
                url = 'https://www.xiaohongshu.com/web_api/sns/v3/page/notes?page_size={}&sort=hot&page_id={}&sid='.format(pagesize, page_id)
            else:
                url = 'https://www.xiaohongshu.com/web_api/sns/v3/page/notes?page_size={}&sort=hot&page_id={}&sid=&cursor={}'.format(pagesize, page_id, next_cursor)

            response = requests.get(url, headers=h1)
            time.sleep(random.uniform(1, timeout_max_sec))

            if response.status_code == 200:
                data = response.json()

                for i in data['data']['notes']:

                    # go to the note page. get data
                    page_url = "https://www.xiaohongshu.com/explore/"+i['id']
                    r = requests.get(page_url, headers=h1)
                    time.sleep(random.uniform(1, timeout_max_sec))
                    if r.status_code == 200:
                        
                        soup = BeautifulSoup(r.text, 'html.parser')

                        comment_icon = soup.select_one('.chat-wrapper span')
                        comment_count = comment_icon.get_text() if comment_icon else 0

                        bookmark_icon = soup.select_one('.collect-wrapper span')
                        bookmarks = bookmark_icon.get_text() if bookmark_icon else 0 

                        content_section = soup.select_one('#detail-desc span')
                        content = content_section.get_text() if content_section else ''

                        #comment not working here, same as tags.
                        # comment_divs = soup.select('div.parent-comment div.content')
                        # for cd in comment_divs:
                        #     if cd:
                        #         cmt = {
                        #             'noteid': i['id'],
                        #             'comment': html.unescape(cd.get_text())
                        #         }
                        #         comments.append(cmt)

                    else:
                        print(f'Error accessing page: Status Code {response.status_code}')


                    if users.get(i['user']['userid'], 0):
                        usr = users[i['user']['userid']]
                    else:
                        #go to the author page, get data
                        user_url = "https://www.xiaohongshu.com/user/profile/"+i['user']['userid']
                        r1 = requests.get(user_url, headers=h1)
                        time.sleep(random.uniform(1, timeout_max_sec))
                        if r1.status_code == 200:
                            
                            soup = BeautifulSoup(r1.text, 'html.parser')

                            intro_div = soup.select_one('.user-desc')
                            intro = intro_div.get_text() if intro_div else ''  

                            user_tags = soup.select('.tag-item')
                            if user_tags:
                                user_tag_str = ' '.join(tag.get_text(strip=True) for tag in user_tags)
                                user_tag_count = len(user_tags)
                            else: 
                                user_tag_str = ''
                                user_tag_count = 0

                            user_data = soup.select('.user-interactions span.count')
                            user_follows = user_data[0].get_text() if user_data else ''
                            user_followers = user_data[1].get_text() if user_data else ''
                            user_like_bookmark_total = user_data[2].get_text() if user_data else ''

                            usr = {
                                'user_intro': intro,
                                'user_tags':user_tag_str,
                                'user_follows':user_follows,
                                'user_followers':user_followers,
                                'user_like_bookmark_total':user_like_bookmark_total
                            }
                            users[i['user']['userid']] = usr

                        else:
                            print(f'Error accessing page: Status Code {response.status_code}')

                    # organize note data
                    row = {    
                        'page': page,
                        'note_url': page_url,
                        'title': i['title'],
                        'noteid': i['id'],
                        'username': i['user']['nickname'],
                        'userid': i['user']['userid'],
                        'create_time': i['create_time'],
                        'likes': i['likes'],
                        'images': i['image_count'],
                        'comments': comment_count,
                        'bookmarks': bookmarks,
                        'content': content,
                        'create_date': TODAY_YMD,
                        'search_keyword': itemtype
                    }
                    row = {**row, **usr}

                    results.append(row)

            else:
                print(f"error happens：{response.status_code}")

            next_cursor = data['data']['cursor']

            # random time interval to mimic human
            time.sleep(random.uniform(1, timeout_max_sec))

            has_more = data['data']['has_more']
            total_cmt = page*pagesize
            if not has_more or (total_cmt >= itemcount): 
                break
            page += 1

    except Exception as e:
        print("error happens：", e)
        
    df_comments = pd.DataFrame(results)
    df_users = pd.DataFrame(users).transpose()
    df_users.columns = ['userid'] + df_users.columns[1:].tolist()

    return df_comments, df_users

In [6]:
# df_guide = hxs_post_scrapper('guide')
# df_hotel = hxs_post_scrapper('hotel')
# df_food = hxs_post_scrapper('food')


# df_tour_cmt, df_tour_user = hxs_post_scrapper('tour')




In [7]:
# # 保存 guide data 到csv
# df_play_cmt, df_play_user = hxs_post_scrapper('guide')
# df_play_cmt.to_csv('./xhs_review_raw_guide_1.csv', mode='a+', index=False, encoding='utf_8_sig')
# df_play_user.to_csv('./xhs_user_raw_guide_1.csv', mode='a+', index=True, encoding='utf_8_sig')

# # 保存 hotel data 到csv
# df_play_cmt, df_play_user = hxs_post_scrapper('hotel')
# df_play_cmt.to_csv('./xhs_review_raw_hotel_1.csv', mode='a+', index=False, encoding='utf_8_sig')
# df_play_user.to_csv('./xhs_user_raw_hotel_1.csv', mode='a+', index=True, encoding='utf_8_sig')

# # 保存 food data 到csv
# df_play_cmt, df_play_user = hxs_post_scrapper('food')
# df_play_cmt.to_csv('./xhs_review_raw_food_1.csv', mode='a+', index=False, encoding='utf_8_sig')
# df_play_user.to_csv('./xhs_user_raw_food_1.csv', mode='a+', index=True, encoding='utf_8_sig')

# # 保存 tour data 到csv
# df_play_cmt, df_play_user = hxs_post_scrapper('tour')
# df_play_cmt.to_csv('./xhs_review_raw_tour_1.csv', mode='a+', index=False, encoding='utf_8_sig')
# df_play_user.to_csv('./xhs_user_raw_tour_1.csv', mode='a+', index=True, encoding='utf_8_sig')

# # 保存 play data 到csv
# df_play_cmt, df_play_user = hxs_post_scrapper('play')
# df_play_cmt.to_csv('./xhs_review_raw_play_1.csv', mode='a+', index=False, encoding='utf_8_sig')
# df_play_user.to_csv('./xhs_user_raw_play_1.csv', mode='a+', index=True, encoding='utf_8_sig')

In [8]:

df_cmt = pd.DataFrame()
df_user = pd.DataFrame()

for key in hxs_topic_page_id_list:
    df_cmt_sub, df_user_sub = hxs_post_scrapper(key, 700, 2)
    df_cmt = pd.concat([df_cmt, df_cmt_sub], ignore_index=True)
    df_user = pd.concat([df_user, df_user_sub], ignore_index=True)
    print(f"data collection done: {key}")

# df_cmt = df_cmt.drop_duplicates(subset='noteid')
# print(f"df cmt shape: {df_cmt.shape}")

# df_user = df_user.drop_duplicates(subset='userid')
# print(f"df user shape: {df_user.shape}")

# df_cmt.to_csv("./xhs_review_raw_"+TODAY_YMD+".csv", mode='a+', index=False, encoding='utf_8_sig')
# df_user.to_csv("./xhs_user_raw+"+TODAY_YMD+".csv", mode='a+', index=True, encoding='utf_8_sig')
    
    

data collection done: guide
data collection done: hotel
data collection done: food
data collection done: tour
data collection done: play
df cmt shape: (3000, 19)


NameError: name 'df_usr' is not defined

In [ ]:
df_cmt = df_cmt.drop_duplicates(subset='noteid')
print(f"df cmt shape: {df_cmt.shape}")

df_user = df_user.drop_duplicates(subset='userid')
print(f"df user shape: {df_user.shape}")

df_cmt.to_csv("./xhs_review_raw_"+TODAY_YMD+".csv", mode='a+', index=False, encoding='utf_8_sig')
df_user.to_csv("./xhs_user_raw+"+TODAY_YMD+".csv", mode='a+', index=True, encoding='utf_8_sig')